In [1]:
# Import relevant libraries
import os       # Os means Open System
import json
import pandas as pd
import traceback
import torch

In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
from langchain import HuggingFaceHub

#find_dotenv()
load_dotenv("c:/Users/andre/mcqgen/.env")

True

In [3]:
KEY=os.getenv("OPENAI_API_KEY")
HG_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
llm = ChatOpenAI(open_ai_key=KEY, model_name="gpt-3.5-turbo", temperature = 0.5)

c:\Users\andre\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! open_ai_key is not default parameter.
                    open_ai_key was transferred to model_kwargs.
                    Please confirm that open_ai_key is what you intended.


In [5]:
import torch

In [6]:
# Import relevant libraries
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
from transformers import pipeline


c:\Users\andre\mcqgen\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Response format that we are passing on the template (as RESPONSE_JSON)
RESPONSE_JSON = {
"1": {
    "mcq": "multiple choice question",
    "option":{
        "a":"choice here",
        "b":"choice here",
        "c":"choice here",
        "d":"choice here",
    },
    "correct":"correct answer",
        
},

"2": {
    "mcq": "multiple choice question",
    "option":{
        "a":"choice here",
        "b":"choice here",
        "c":"choice here",
        "d":"choice here",
    },
    "corrctect":"correct answer",

},

"3": {
    "mcq": "multiple choice question",
    "option":{
        "a":"choice here",
        "b":"choice here",
        "c":"choice here",
        "d":"choice here",
    },
    "corrctect":"correct answer"

},

}

In [8]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is yout job to \
create a quiz of {number} multiple coice questions for {subject} students in {tone} tone.
make sure the questions are nor repeate and check all teh questions to ve conforming the text as well.
MAke sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensrude to make {number} MCQs
### REPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'number', 'response_json'],
    template = TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm = HuggingFaceHub(repo_id= 'openai-community/gpt2'), prompt = quiz_generation_prompt, output_key="quiz", verbose = True)

c:\Users\andre\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
c:\Users\andre\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
TEMPLATE2 = """
You are an expert grammarian and write. Give me a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 word for complexity. 
If the quiz is not at per with cognitive and analytical abilities of the students.\.
Update the quiz questions which needs to be changed and change the tone that it perfectly fits the students  abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(input_variables = [ "subject", "quiz"], template = TEMPLATE)

In [13]:
review_chain = LLMChain(llm = HuggingFaceHub(repo_id= 'openai-community/gpt2'), prompt = quiz_evaluation_prompt, output_key="review", verbose = True)

In [14]:
generate_evaluate_chain = SequentialChain(chains= [quiz_chain, review_chain], input_variables=['text', 'number', 'subject', 'tone', 'number', 'response_json'], output_variables=["quiz", "review"], verbose= True)

In [15]:
file_path = r"C:\Users\andre\mcqgen\data.txt" 

In [16]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [17]:
# Serialize the Pyhton dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "corrctect": "correct answer"}, "3": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "corrctect": "correct answer"}}'

In [18]:
NUMBER = 3
SUBJECT = "machine learning"
TONE="simple"

In [19]:
response = generate_evaluate_chain(
    {
        "text" : TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)

    }                                   
                                   )

c:\Users\andre\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]
You are an expert MCQ maker. Given the above text, it is yout job to create a quiz of 3 multiple coice questions for machine learning students in simple tone.
make sure the questions are nor repeate and check all teh questions to ve conforming the text as well.
MAke sure to format your response like RESPONSE_JSON below and use it as a guide. Ensrude to make 3 MCQs
### REPONSE_JSON
{"1": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice he

In [20]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]',
 'number': 3,
 'subject': 'machine learning',
 'tone': 'simple',
 'response_json': '{"1": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "corrctect": "correct answer"}, "3": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "corrctect": "correct answer"}}',
 'quiz': '\nText:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8]